# Search for AGN in the Extended Chandra Deep Field South

- Confluence page : https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/372867091/System-level+Science+Performance+Verification+Sprint
- slides : https://docs.google.com/presentation/d/1bPjS5NGtiEL2lfjmvP1UYdG_tMEDfZxX26ayhf7MhtY/edit#slide=id.ga2f7fb1a1f_0_70

- where to find the campains
- https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

- plot Navigator
- https://usdf-rsp.slac.stanford.edu/plot-navigator
- https://usdf-rsp.slac.stanford.edu/plot-navigator/plot/%2Frepo%2Fmain/LSSTComCam%2Fruns%2FDRP%2FDP1%2Fw_2025_05%2FDM-48666/objectTableCore_coaddInputCount_SkyPlot

- Notebooks examples
- https://github.com/lsst-dm/DMTR-401/blob/main/notebooks/test_LVV-T40_T1240.ipynb
- https://github.com/lsst-dm/DMTR-412/blob/tickets/DM-38728/notebooks/test_LVV-T1751_AM1_AM2.ipynb

- author : Sylvie Dagoret-Campagne
- creattion date : 2025-02-13
- update : 2025-02-19
- last update : 2025-04-21
- Redo Visits like here : https://github.com/sylvielsstfr/LSST-Rehearsal2024/blob/main/notebooks/Visits/stat_on_visits_LSSTComCamSim.ipynb
- To find what I did on LSSTComCamSim : https://github.com/sylvielsstfr/LSST-Rehearsal2024/blob/main/notebooks/LightCurves/MultiColor_lightCurves-DMRehearsal2024_01-AuxTel-DZPOnCCD.ipynb
- **Confluence page** : https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/443613290/Science+Verification+Sprint+Feb+2025


In [ ]:
# Confirm that the version of the Science Pipelines is recent:
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

In [ ]:
from lsst.daf.butler import Butler
import lsst.geom as geom
from lsst.geom import SpherePoint, degrees

In [ ]:
import os
import gc
import glob
import numpy as np
import pandas as pd
import random

import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"  # Reselect Data Release 3, default
Gaia.ROW_LIMIT = 100000
from astropy.visualization import (MinMaxInterval, AsinhStretch, ZScaleInterval, LogStretch, LinearStretch,
                                   ImageNormalize)

In [ ]:
import seaborn as sns
from itertools import cycle, islice

In [ ]:
from astropy.time import Time
from datetime import datetime, timedelta

In [ ]:
import lsst
import lsst.geom as geom
from lsst.geom import Angle
from lsst.geom import SpherePoint
from lsst.geom import AngleUnit

In [ ]:
# https://pipelines.lsst.io/modules/lsst.geom/getting-started.html
func_degToRad = lambda x : Angle(x,lsst.geom.degrees).asRadians()

In [ ]:
# Set plotting defaults
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.colors import ListedColormap
from matplotlib import colors
zscale = ZScaleInterval()

# Set up some plotting defaults:
plt.rcParams.update({'figure.figsize' : (12, 8)})
plt.rcParams.update({'font.size' : 24})
plt.rcParams.update({'axes.linewidth' : 3})
plt.rcParams.update({'axes.labelweight' : 3})
plt.rcParams.update({'axes.titleweight' : 5})
plt.rcParams.update({'ytick.major.width' : 3})
plt.rcParams.update({'ytick.minor.width' : 2})
plt.rcParams.update({'ytick.major.size' : 8})
plt.rcParams.update({'ytick.minor.size' : 5})
plt.rcParams.update({'xtick.major.size' : 8})
plt.rcParams.update({'xtick.minor.size' : 5})
plt.rcParams.update({'xtick.major.width' : 3})
plt.rcParams.update({'xtick.minor.width' : 2})
plt.rcParams.update({'xtick.direction' : 'in'})
plt.rcParams.update({'ytick.direction' : 'in'})


In [ ]:
def angle_in_range(alpha, lower, upper):
    return (alpha - lower) % 360 <= (upper - lower) % 360

In [ ]:
def get_bbox_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    ramin,ramax,decmin,decmax in decimal degrees
    """

    xmin = bbox.beginX
    xmax = bbox.endX
    ymin = bbox.beginY
    ymax = bbox.endY
    
    radec_ll = wcs.pixelToSky(xmin, ymin)
    radec_ur = wcs.pixelToSky(xmax, ymax)
        
    return radec_ur.getRa().asDegrees(),radec_ll.getRa().asDegrees(), radec_ll.getDec().asDegrees(),radec_ur.getDec().asDegrees() 


In [ ]:
def isradec_inbbox_angularsep(ra,dec,wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    ra: ra in degree
    dec: dec in degree
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    float in degree
    
    """

   
    RAMIN,RAMAX,DECMIN,DECMAX = get_bbox_radec(wcs, bbox)
  
    RAMEAN = np.mean([RAMIN,RAMAX])
    DECMEAN = np.mean([DECMIN,DECMAX])

    sp0 = SpherePoint(longitude=geom.Angle(ra,geom.degrees),latitude=geom.Angle(dec,geom.degrees))
    sp1 = SpherePoint(longitude=geom.Angle(RAMEAN,geom.degrees),latitude=geom.Angle(DECMEAN,geom.degrees)) 
    
    sep = sp0.separation(sp1).asDegrees()
     
    return sep


In [ ]:
def isradec_inbbox_radec(ra,dec,wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    ra: ra in degree
    dec: dec in degree
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    Bool
    
    """

    RAMIN,RAMAX,DECMIN,DECMAX = get_bbox_radec(wcs, bbox)

    flag_ra = angle_in_range(ra,RAMIN,RAMAX)
    flag_dec = angle_in_range(dec,DECMIN,DECMAX)

    flag = flag_ra and flag_dec
    return flag


In [ ]:
def FindTractAndPatch(row):
    """
    Apply this function on ccdvisitTable dataframe to find the tract and patch for each visit
    """
     
    try:
        ra = row["ra"]
        dec = row["dec"]
        selectFlag = False
        # loop on tract
        for tractID in tractsId_list:
            tractInfo = skymap.generateTract(tractID)
            patches_selected = [patch.getSequentialIndex() for patch in tractInfo]
            wcs=tractInfo.getWcs()
            # loop on patches
            for patch in tractInfo:
                patchID = patch.getSequentialIndex()
                if patchID in patches_selected:
                    ibb=patch.getInnerBBox()
                    flag = isradec_inbbox_radec(ra,dec,wcs, ibb)
                    selectFlag =  selectFlag or flag
                    if selectFlag:
                        return pd.Series([tractID,patchID])
        return pd.Series([0,0])           
                
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)          # __str__ allows args to be printed directly,
        return pd.Series([0,0])  
        

In [ ]:
def FindTractAndPatchFromAngularSep(row):
    """
    Apply this function on ccdvisitTable dataframe to find the tract and patch for each visit
    Need the tractsId_list not to scan the whole skymap
    """

     
    try:
        ra = row["ra"]
        dec = row["dec"]
        
        # loop on tracts
        List_of_tractids = []
        List_of_patchids = []
        List_of_sep = []
        
        for tractID in tractsId_list:
            tractInfo = skymap.generateTract(tractID)
            patches_selected = [patch.getSequentialIndex() for patch in tractInfo]
         
            wcs=tractInfo.getWcs()
            # loop on patches
            patches_ids = []
            patches_sep = []
            # loop on patches
            for patch in tractInfo:
                patchID = patch.getSequentialIndex()
                if patchID in patches_selected:
                    ibb=patch.getInnerBBox()
                  
                    sep = isradec_inbbox_angularsep(ra,dec,wcs, ibb)
                    patches_ids.append(patchID)
                    patches_sep.append(sep)
                    
            # find the patch with the minimum distance
            patches_ids=np.array(patches_ids)
            patches_sep= np.array(patches_sep)
            idx_sepmin = int(np.where(patches_sep==patches_sep.min())[0])
            
            
            List_of_tractids.append(tractID)
            List_of_patchids.append(patches_ids[idx_sepmin])
            List_of_sep.append(patches_sep[idx_sepmin])
            
        List_of_tractids=np.array(List_of_tractids)
        List_of_patchids=np.array(List_of_patchids)
        List_of_sep= np.array(List_of_sep)
        idx_sepmin = int(np.where(List_of_sep==List_of_sep.min())[0])

        tractID_sel = List_of_tractids[idx_sepmin]
        patchID_sel = List_of_patchids[idx_sepmin]
       
            
        return pd.Series([tractID_sel,patchID_sel])
               
                
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)          # __str__ allows args to be printed directly,
        return pd.Series([0,0])  
        

In [ ]:
def plotAVisit(row,ax=None):
    """
    row : dataframe row
    ax 
    """

    if ax == None:
        fig, ax = plt.subplots(1,1,figsize=(5,5))
        
    visitid = row["visitId"]
    ra= row["ra"]
    dec = row["dec"]
    tract_id  = row["tractID"]
    patch_id  = row["patchID"]
    label = f"tract {tract_id} , patch {patch_id}"              
    
    tractInfo = skymap.generateTract(tract_id)
    patches_selected = [patch.getSequentialIndex() for patch in tractInfo]     
    wcs=tractInfo.getWcs()
            
    # loop on patches
    all_ra = []
    all_dec = []
    corners = []
    for patch in tractInfo:
        patchID = patch.getSequentialIndex()
        if patchID == patch_id :
            ibb=patch.getInnerBBox()
            RAMIN,RAMAX,DECMIN,DECMAX = get_bbox_radec(wcs, ibb)
            for icorn,corner in enumerate(ibb.getCorners()):
                p = geom.Point2D(corner.getX(), corner.getY())
                coord = wcs.pixelToSky(p)
                corners.append([coord.getRa().asDegrees(), coord.getDec().asDegrees()])
                all_ra.append(coord.getRa().asDegrees()) 
                all_dec.append(coord.getDec().asDegrees()) 
    if (len(all_ra)>0) and (len(all_dec)>0):
        all_ra.append(all_ra[0])
        all_dec.append(all_dec[0])
        print(ax)
        ax.plot(all_ra,all_dec,'b-',lw=3,label=label)
        ax.scatter([ra],[dec],marker = 'o',s=20,c="r",label=visitid)
        ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        ax.legend()
                

## Build the table of AGN

In [ ]:
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
import astropy.units as u

In [ ]:
ECDFS_RA = 53.1
ECDFS_DEC = -27.8
ECDFS_COORD = SpherePoint(ECDFS_RA, ECDFS_DEC , degrees)

In [ ]:
# Coordonnées approximatives du centre du E-CDFS
coord = SkyCoord(ra=ECDFS_RA, dec = ECDFS_DEC, unit=(u.deg, u.deg), frame='icrs')
radius = 15 * u.arcmin

# Exemple avec le catalogue Luo+2017 (7Ms CDF-S)
catalog_id = "J/ApJS/228/2"
result = Vizier(columns=["*"]).query_region(coord, radius=radius, catalog=catalog_id)

In [ ]:
sources_table = result['J/ApJS/228/2/sources']
print(sources_table.colnames)

In [ ]:
# On calcule le log10 de LX
logLX = np.log10(sources_table['LX'])

# Et on l'ajoute comme nouvelle colonne à la table
sources_table['logLX'] = logLX

# Attention au nom exact de la colonne
logLX = sources_table['logLX']
is_agn = logLX > 42

is_agn = sources_table['logLX'] > 42  # AGN si LX > 10^42 erg/s
agn_table = sources_table[is_agn]

In [ ]:
agn_table[['RAJ2000', 'DEJ2000', 'logLX', 'LX', 'zspec']].pprint(max_lines=10)

In [ ]:
galaxy_table = sources_table[sources_table['OType'] == 'Galaxy']

In [ ]:
galaxy_table[['RAJ2000', 'DEJ2000', 'logLX', 'LX', 'zspec']].pprint(max_lines=10)

## Start Here with initialisation


- Check here the collection available : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

In [ ]:
# The output repo is tagged with the Jira ticket number "DM-40356":
repo = '/repo/main'
#collection = 'LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666' # work
#collection = 'LSSTComCam/runs/DRP/DP1/w_2025_06/DM-48810' # work
collection = 'LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359' # work


# bad : crash collection = 'LSSTComCam/runs/DRP/DP1/w_2025_08/DM-49029'

# bad : collection = "LSSTComCam/runs/DRP/20241101_20241211/w_2024_51/DM-48233"

# not working perhaps because I am using w_2025_10 version
# bad : no ccd visit collection = "LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864"
# bad : no ccd visit collection = 'LSSTComCam/runs/DRP/DP1/w_2025_15/DM-50050'
# bad : no cce visit collection = 'LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864'
# bad : no cce visit collection collection = 'LSSTComCam/runs/DRP/DP1/w_2025_13/DM-49751'


instrument = "LSSTComCam"
skymapName = "lsst_cells_v1"
where_clause = "instrument = \'" + instrument+ "\'"
collectionStr = collection.replace("/", "_")
NDET = 9
TRACTSEL = 5063
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection)
registry = butler.registry

In [ ]:
skymap = butler.get('skyMap', skymap=skymapName, collections=collection)

In [ ]:
# 4. Trouver dans quel tract/patch se trouve la coordonnée
ECDFS_tract_info = skymap.findTract(ECDFS_COORD)

In [ ]:
print('Tract number for ECDFS ::',ECDFS_tract_info.getId())

In [ ]:
TRACTSEL = ECDFS_tract_info.getId()

In [ ]:
# Check here the collections available
#
for _ in registry.queryCollections():
    if collection in _:
        print(_)

In [ ]:
# List a number of usefull data-product to explore
if 0:
    for datasetType in registry.queryDatasetTypes():
        if registry.queryDatasets(datasetType, collections=collection).any(
            execute=False, exact=False
        ):
            # Limit search results to the data products
            if (
                ("_config" not in datasetType.name)
                and ("_log" not in datasetType.name)
                and ("_metadata" not in datasetType.name)
                and ("_resource_usage" not in datasetType.name)
                and (("Table" in datasetType.name) or ("Zeropointp" in datasetType.name) or ("fgcm" in datasetType.name) or ("transm" in datasetType.name) or ("Transm" in datasetType.name)
                or ("source" in datasetType.name) or ("Source" in datasetType.name) or ("object" in datasetType.name) or ("Object" in datasetType.name))
            ):
                print(datasetType)

## Get list of Tracts and Patches

In [ ]:
datasettype = "objectTable_tract"
therefs = butler.registry.queryDatasets(datasettype,  collections=collection)

In [ ]:
tractsId_list = np.unique([ref.dataId['tract'] for ref in therefs])
tractsId_list = sorted(tractsId_list)
print(tractsId_list)

## Forced source table

In [ ]:
df_galaxy = galaxy_table.to_pandas()

In [ ]:
df_agn = agn_table.to_pandas()

In [ ]:
df_agn

In [ ]:
plt.figure(figsize=(14,5))

plt.subplot(1, 2, 1)
plt.hist(df_agn['zspec'].values, bins=20, color='green', alpha=0.7)
plt.xlabel("Redshift zspec")
plt.ylabel("Nombre de sources")
plt.title("Distribution en redshift")

plt.subplot(1, 2, 2)
plt.hist(df_agn['logLX'].values, bins=20, color='orange', alpha=0.7)
plt.xlabel("log(Lx) [erg/s]")
plt.ylabel("Nombre de sources")
plt.title("Distribution de la luminosité X")

plt.tight_layout()
plt.show()


## Select One AGN

In [ ]:
df_agn[df_agn["zspec"]<1.5]

In [ ]:
galaxyindex_selected = []
N =len(df_galaxy)
for idx in range(N):
    ra_target,dec_target = df_galaxy.iloc[idx][['RAJ2000','DEJ2000']]
    target_point = SpherePoint(ra_target, dec_target, degrees)
    if ECDFS_tract_info.contains(target_point):
        galaxyindex_selected.append(idx)         

In [ ]:
print(galaxyindex_selected)

In [ ]:
agnindex_selected = []
N =len(df_agn)
for idx in range(N):
    ra_target,dec_target = df_agn.iloc[idx][['RAJ2000','DEJ2000']]
    target_point = SpherePoint(ra_target, dec_target, degrees)
    if ECDFS_tract_info.contains(target_point):
        agnindex_selected.append(idx)         

In [ ]:
print(agnindex_selected)

In [ ]:
index_agn_selected = 0
index_galaxy_selected = 0
FLAG_AGN_SELECTED  = False

In [ ]:
if FLAG_AGN_SELECTED: 
    ra_target,dec_target = df_agn.iloc[index_agn_selected ][['RAJ2000','DEJ2000']]
    target_point = SpherePoint(ra_target, dec_target, degrees)
    target_coord = SkyCoord(ra=ra_target*u.deg, dec=dec_target*u.deg)
    logLx = df_agn.iloc[index_agn_selected ]['logLX']
    zspec = df_agn.iloc[index_agn_selected ]['zspec']
    target_title = f"agn :: {index_agn_selected}, log L(erg/s) = {logLx:.2f} , zspec = {zspec:.2f}, (ra,dec) = {ra_target:.5f},{dec_target:.5f}"
else:
    ra_target,dec_target = df_galaxy.iloc[index_galaxy_selected ][['RAJ2000','DEJ2000']]
    target_point = SpherePoint(ra_target, dec_target, degrees)
    target_coord = SkyCoord(ra=ra_target*u.deg, dec=dec_target*u.deg)
    logLx = df_galaxy.iloc[index_galaxy_selected ]['logLX']
    zspec = df_galaxy.iloc[index_galaxy_selected ]['zspec']
    target_title = f"galaxy :: {index_galaxy_selected}, log L(erg/s) = {logLx:.2f} , zspec = {zspec:.2f}, (ra,dec) = {ra_target:.5f},{dec_target:.5f}"
    

In [ ]:
datasettype =  'forcedSourceOnDiaObjectTable'
#datasettype =  'diaSourceTable_tract'
#datasettype = 'goodSeeingDiff_assocSsSrcTable' # bad table
therefs = butler.registry.queryDatasets(datasettype,  collections=collection)

In [ ]:
if FLAG_AGN_SELECTED: 
    figname = f"agn_{index_agn_selected}_{datasettype}.png"
else:
    figname = f"galaxy_{index_agn_selected}_{datasettype}.png"

In [ ]:
all_tables = []

for count,ref in enumerate(therefs):
    the_id = ref.dataId
    the_tract_id = the_id["tract"] 
    # select the visits in that Tract (ECDFS)
    if the_tract_id == TRACTSEL: 
        catalog = butler.get(ref)
        #catalog["dateobs"] = catalog.apply(lambda x: x['visit']//100_000, axis=1)
        #catalog["seq"] = catalog["visit"] - catalog["dateobs"]*100_000
        #t["ra_rad"] = t["coord_ra"].apply(func_degToRad)  
        #t["ra_dec"] = t["coord_dec"].apply(func_degToRad)  
        ra = catalog["coord_ra"].values
        dec = catalog["coord_dec"].values
        # Créer tous les points du catalogue
        #points = [SpherePoint(r, d, degrees) for r, d in zip(ra, dec)]
        catalog_coords = SkyCoord(ra=ra*u.deg, dec=dec*u.deg)

        # Calculer la distance angulaire à la cible (en arcsec)
        #distances_arcsec = np.array([pt.separation(target_point).asArcseconds() for pt in points])
        # 5. Calcul vectorisé des distances (résultat en arcsec)
        distances_arcsec = target_coord.separation(catalog_coords).arcsecond
        catalog["sep"] = distances_arcsec

        # 5. Sélectionner les objets proches (ex: < 1.5 arcsec)
        if datasettype ==  'forcedSourceOnDiaObjectTable':
            radius_arcsec = 3
        elif datasettype ==  'diaSourceTable_tract':
            radius_arcsec = 5
        elif datasettype == 'goodSeeingDiff_assocSsSrcTable':
            radius_arcsec = 5
        else:
            radius_arcsec = 3

        # select
        nearby = catalog[distances_arcsec < radius_arcsec]
        
        if len(nearby) > 0 :
            #print(nearby)
            all_tables.append(nearby)  
            #break

        #if(len(all_tables)>10):
        #    break
   

In [ ]:
len(all_tables)

In [ ]:
t = all_tables[0]

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,4))
t["sep"].hist(bins=50,ax=ax)
plt.show()

In [ ]:
t = t[t["sep"]<3.5]

### Split visit into dayobs and seq

In [ ]:
t["dateobs"] = t.apply(lambda x: x['visit']//100_000, axis=1)
t["seq"] = t["visit"] - t["dateobs"]*100_000

In [ ]:
t["seq"]

In [ ]:
t["mags"] = t["psfFlux"].apply(lambda flux : -2.5 * np.log10(flux) + 31.4)
t["valid"] = (t["psfFlux"] >0) & (~t["psfFlux_flag"])
visit_list = t["visit"].astype(int)

#### Find the time associated to a visit

In [ ]:
# On interroge la table visitDefinition
rows = registry.queryDimensionRecords("visit", where=f"visit in {tuple(visit_list)}")

# 4. Construire un tableau des résultats
results = []
for row in rows:
    visit_id = row.id

    # Extraire l'instant de début de l'observation (Time astropy)
    start_time = row.timespan.begin

    # Convertir en MJD et ISO
    mjd = start_time.to_value("mjd")  # Ex: 60384.28718
    isot = start_time.to_value("isot")  # Ex: '2024-04-19 06:53:32.000'
    
    #mjd = row.startDate.toMjd()
    #utc = Time(mjd, format='mjd', scale='utc').to_value('iso')
    results.append({"visit": visit_id, "mjd": mjd, "isot": isot})

df_times = pd.DataFrame(results).sort_values("visit")
df_times.set_index("visit",inplace=True)

In [ ]:
t["mjd"] = t["visit"].apply(lambda x: df_times.loc[x]["mjd"])
t["isot"] = t["visit"].apply(lambda x: df_times.loc[x]["isot"])

In [ ]:
tb = {}
for band in ["u", "g", "r", "i", "z", "y"]:
    tb[band] = t[t["band"] == band]

In [ ]:
band_to_col = {"u":"b", "g":"g", "r":"r", "i":"orange", "z":"grey", "y":"k"}

In [ ]:
fig,axs = plt.subplots(6,1,figsize=(18,16),sharex=True,layout="constrained")

for idx,band in enumerate(["u", "g", "r", "i", "z", "y"]):
    ax =axs[idx]
    ax.scatter(tb[band].mjd, tb[band].mags,color=band_to_col[band])
    ax.set_ylabel("mag_psfFlux")
    if idx==0:
        ax.set_title(target_title,fontweight = "bold")
ax.set_xlabel("mjd")

subtitle = collectionStr + "_" + datasettype
plt.suptitle(subtitle)
#plt.tight_layout()
plt.savefig(figname)
plt.show()